## Sanity Test

In [ ]:
import h5py
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pennylane as qml

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and preprocess data
jet_images_path = 'jet-images_Mass60-100_pT250-300_R1.25_Pix25.hdf5'
with h5py.File(jet_images_path, 'r') as jet_mass_data:
    X_jet = jet_mass_data['image']
    print("Original shape:", X_jet.shape)
    X_jet = np.array(X_jet)
    X_jet = torch.tensor(X_jet, dtype=torch.float32)

# Normalize and resize images
X_jet = (X_jet - X_jet.min()) / (X_jet.max() - X_jet.min())
X_jet = X_jet.unsqueeze(1)
X_jet_resized = nn.functional.interpolate(X_jet, size=(16, 16), mode='bilinear', align_corners=False)
print("Resized shape:", X_jet_resized.shape)

# Sample data
indices = random.sample(range(X_jet_resized.shape[0]), 10000)
X_jet_sampled = X_jet_resized[indices]
print("Sampled shape:", X_jet_sampled.shape)

# Flatten images and apply PCA
X_flat = X_jet_sampled.view(-1, 256).numpy()
n_components = 8
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_flat)

# Create dataset and dataloader
class JetDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

dataset = JetDataset(X_pca)
batch_size = 64
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define quantum devices

Original shape: (872666, 25, 25)
Resized shape: torch.Size([872666, 1, 16, 16])
Sampled shape: torch.Size([10000, 1, 16, 16])


In [ ]:
@qml.qnode(dev_generator, interface='torch')
def generator_qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

@qml.qnode(dev_discriminator, interface='torch')
def discriminator_qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# Define weight shapes for the models
generator_weight_shapes = {"weights": (n_layers, n_qubits, 3)}
discriminator_weight_shapes = {"weights": (n_layers, n_qubits, 3)}

# Create the generator and discriminator models
generator = qml.qnn.TorchLayer(generator_qnode, generator_weight_shapes).to(device)
discriminator = qml.qnn.TorchLayer(discriminator_qnode, discriminator_weight_shapes).to(device)

# Define optimizers and loss function
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.01)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.01)
criterion = nn.BCELoss()

# Training loop
num_epochs = 30
d_losses = []
g_losses = []

In [ ]:
for epoch in range(num_epochs):
    for i, real_data in enumerate(dataloader):
        batch_size = real_data.size(0)
        real_data = real_data.to(device)

        # Train Discriminator
        d_optimizer.zero_grad()
        # Real data
        real_labels = torch.ones(batch_size).to(device)  # Changed shape to [batch_size]
        real_output = discriminator(real_data)
        real_output = (real_output + 1)/2  # Map to [0,1]
        d_loss_real = criterion(real_output, real_labels)

        # Fake data
        noise = torch.randn(batch_size, n_qubits).to(device)
        fake_data = generator(noise)
        fake_data = fake_data.detach()
        fake_output = discriminator(fake_data)
        fake_output = (fake_output + 1)/2  # Map to [0,1]
        fake_labels = torch.zeros(batch_size).to(device)  # Changed shape to [batch_size]
        d_loss_fake = criterion(fake_output, fake_labels)

        # Total discriminator loss
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        g_optimizer.zero_grad()
        noise = torch.randn(batch_size, n_qubits).to(device)
        fake_data = generator(noise)
        fake_output = discriminator(fake_data)
        fake_output = (fake_output + 1)/2  # Map to [0,1]
        g_loss = criterion(fake_output, real_labels)  # Try to fool discriminator
        g_loss.backward()
        g_optimizer.step()

        # Save losses
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

## Perceptual-Quantum Loss

Since we are working with quantum data, we want to balance two key objectives during training:

1. **Image Quality**: Ensuring that the generated images are perceptually similar to real images.
2. **Quantum Fidelity**: Ensuring that the quantum states of the generated data resemble those of the real data.

To achieve this balance, we introduce a combined loss function known as **Perceptual Quantum Loss**.

#### **1. Perceptual Loss**

The **Perceptual Loss** is used to measure the similarity between real and generated images based on high-level features extracted from a pre-trained network (like **VGG16**). Instead of comparing images at the pixel level, the perceptual loss compares features from deeper layers of the network, capturing more abstract patterns like edges, textures, and shapes. This ensures that the generated images are perceptually similar to real ones.

The pre-trained network (VGG16) acts as a "feature extractor" for this comparison. We resize the generated images and real images to match the input size of VGG16, extract their features, and compute the mean squared error (MSE) between these features. Lower perceptual loss indicates that the generated images look more like the real images.

#### **2. Quantum Fidelity**

Since we are working with quantum data, we also need to ensure that the quantum states of the generated data closely match those of the real data. **Quantum Fidelity** measures the similarity between two quantum states. Higher fidelity means that the quantum states are more similar, indicating that the generator is learning to produce data that is not only visually accurate but also quantum-accurate.

In our implementation, we encode both real and generated data into quantum circuits and compute the fidelity between the quantum states.

#### **3. Combined Loss Function**

The **Perceptual Quantum Loss** combines these two components:

$$
L_{\text{total}} = L_{\text{adversarial}} + \alpha \times L_{\text{perceptual}} - \beta \times L_{\text{fidelity}}
$$

- **Adversarial Loss (`L_adversarial`)**: This is the standard GAN loss that drives the generator to produce images that fool the discriminator.
- **Perceptual Loss (`L_perceptual`)**: Encourages the generator to produce images that are perceptually similar to real images.
- **Quantum Fidelity (`L_fidelity`)**: Ensures that the quantum states of generated data match those of the real data.

Here, `alpha` controls the importance of the perceptual loss, and `beta` controls the importance of quantum fidelity. By adjusting these hyperparameters, we can fine-tune the balance between visual similarity and quantum state similarity.

#### **Why Use Perceptual Quantum Loss?**

In Quantum GANs, we aim to generate data that is accurate in two domains:
- **Visual domain**: Ensuring that the images look like real images.
- **Quantum domain**: Ensuring that the quantum data respects the underlying quantum structure.

Perceptual Quantum Loss allows us to address both goals in a single loss function, making it a powerful tool for training Quantum GANs to generate realistic data while maintaining quantum properties.


In [ ]:
import h5py
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pennylane as qml
from torchvision import models, transforms
from skimage.metrics import structural_similarity as ssim
import pandas as pd

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Load and preprocess data
# jet_images_path = 'jet-images_Mass60-100_pT250-300_R1.25_Pix25.hdf5'
# with h5py.File(jet_images_path, 'r') as jet_mass_data:
#     X_jet = jet_mass_data['image']
#     print("Original shape:", X_jet.shape)
#     X_jet = np.array(X_jet)
#     X_jet = torch.tensor(X_jet, dtype=torch.float32)

# # Normalize and resize images
# X_jet = (X_jet - X_jet.min()) / (X_jet.max() - X_jet.min())
# X_jet = X_jet.unsqueeze(1)
# X_jet_resized = nn.functional.interpolate(X_jet, size=(16, 16), mode='bilinear', align_corners=False)
# print("Resized shape:", X_jet_resized.shape)

# # Sample data
# indices = random.sample(range(X_jet_resized.shape[0]), 10000)
# X_jet_sampled = X_jet_resized[indices]
# print("Sampled shape:", X_jet_sampled.shape)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load and preprocess data
jet_images_path = '/content/drive/MyDrive/jet-images_Mass60-100_pT250-300_R1.25_Pix25.hdf5'


In [ ]:
with h5py.File(jet_images_path, 'r') as jet_mass_data:
    X_jet = jet_mass_data['image']
    print("Original shape:", X_jet.shape)
    X_jet = np.array(X_jet)
    X_jet = torch.tensor(X_jet, dtype=torch.float32)

# Normalize and resize images
X_jet = (X_jet - X_jet.min()) / (X_jet.max() - X_jet.min())
X_jet = X_jet.unsqueeze(1)
X_jet_resized = nn.functional.interpolate(X_jet, size=(16, 16), mode='bilinear', align_corners=False)
print("Resized shape:", X_jet_resized.shape)

# Sample data
indices = random.sample(range(X_jet_resized.shape[0]), 10000)
X_jet_sampled = X_jet_resized[indices]
print("Sampled shape:", X_jet_sampled.shape)

In [ ]:
# Flatten images and apply PCA
X_flat = X_jet_sampled.view(-1, 256).numpy()
n_components = 8
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_flat)

# Create dataset and dataloader
class JetDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

dataset = JetDataset(X_pca)
batch_size = 64
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define quantum devices
n_qubits = n_components
n_layers = 3
dev_generator = qml.device('default.qubit', wires=n_qubits)
dev_discriminator = qml.device('default.qubit', wires=n_qubits)

# Define the generator and discriminator quantum circuits
@qml.qnode(dev_generator, interface='torch')
def generator_qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

@qml.qnode(dev_discriminator, interface='torch')
def discriminator_qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))


In [ ]:
# Define weight shapes for the models
generator_weight_shapes = {"weights": (n_layers, n_qubits, 3)}
discriminator_weight_shapes = {"weights": (n_layers, n_qubits, 3)}

# Create the generator and discriminator models
generator = qml.qnn.TorchLayer(generator_qnode, generator_weight_shapes).to(device)
discriminator = qml.qnn.TorchLayer(discriminator_qnode, discriminator_weight_shapes).to(device)

# Define optimizers and loss function
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.01)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.01)
criterion = nn.BCELoss()

# Load pre-trained VGG16 model for perceptual loss
vgg = models.vgg16(pretrained=True).features[:8].eval().to(device)
# Freeze VGG parameters
for param in vgg.parameters():
    param.requires_grad = False

# Define image preprocessing for VGG
vgg_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # Normalization parameters for VGG
])

# Function to compute perceptual loss
def perceptual_loss(real_images, fake_images):
    # Both inputs are in PCA space, need to inverse transform and reshape
    real_images = pca.inverse_transform(real_images.cpu().numpy())
    fake_images = pca.inverse_transform(fake_images.cpu().detach().numpy())

    real_images = torch.tensor(real_images, dtype=torch.float32).view(-1, 1, 16, 16).to(device)
    fake_images = torch.tensor(fake_images, dtype=torch.float32).view(-1, 1, 16, 16).to(device)

    # Upsample images to 224x224 and replicate channels to match VGG input
    real_images = F.interpolate(real_images, size=(224, 224), mode='bilinear', align_corners=False)
    fake_images = F.interpolate(fake_images, size=(224, 224), mode='bilinear', align_corners=False)
    real_images = real_images.repeat(1, 3, 1, 1)  # Convert to 3 channels
    fake_images = fake_images.repeat(1, 3, 1, 1)

    # Normalize images
    real_images = vgg_transform(real_images)
    fake_images = vgg_transform(fake_images)

    # Extract features
    real_features = vgg(real_images)
    fake_features = vgg(fake_images)

    # Compute perceptual loss
    loss = F.mse_loss(fake_features, real_features)
    return loss

# Function to compute quantum fidelity
# Function to compute quantum fidelity
def quantum_fidelity(real_data, fake_data):
    # real_data and fake_data are in PCA space
    # Define a QNode that outputs the state vector
    dev_fidelity = qml.device('default.qubit', wires=n_qubits)

    @qml.qnode(dev_fidelity, interface='torch')
    def state_preparation(inputs):
        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        return qml.state()

    real_states = [state_preparation(data).cpu().numpy() for data in real_data] # Move tensors to CPU before converting to NumPy
    fake_states = [state_preparation(data).cpu().detach().numpy() for data in fake_data] # Move tensors to CPU before converting to NumPy and detach from the computation graph

    fidelities = []
    for real_state, fake_state in zip(real_states, fake_states):
        fidelity = np.abs(np.vdot(real_state, fake_state)) ** 2
        fidelities.append(fidelity)
    return np.mean(fidelities)

# Training loop
num_epochs = 30
d_losses = []
g_losses = []
perceptual_losses = []
fidelities = []

In [ ]:

for epoch in range(num_epochs):
    for i, real_data in enumerate(dataloader):
        batch_size = real_data.size(0)
        real_data = real_data.to(device)

        # Train Discriminator
        d_optimizer.zero_grad()
        # Real data
        real_labels = torch.ones(batch_size).to(device)
        real_output = discriminator(real_data)
        real_output = (real_output + 1)/2  # Map to [0,1]
        d_loss_real = criterion(real_output, real_labels)

        # Fake data
        noise = torch.randn(batch_size, n_qubits).to(device)
        fake_data = generator(noise)
        fake_data = fake_data.detach()
        fake_output = discriminator(fake_data)
        fake_output = (fake_output + 1)/2  # Map to [0,1]
        fake_labels = torch.zeros(batch_size).to(device)
        d_loss_fake = criterion(fake_output, fake_labels)

        # Total discriminator loss
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        g_optimizer.zero_grad()
        noise = torch.randn(batch_size, n_qubits).to(device)
        fake_data = generator(noise)
        fake_output = discriminator(fake_data)
        fake_output = (fake_output + 1)/2  # Map to [0,1]
        g_loss = criterion(fake_output, real_labels)  # Try to fool discriminator

        # Compute Perceptual Loss
        ploss = perceptual_loss(real_data, fake_data)

        # Compute Quantum Fidelity
        q_fidelity = quantum_fidelity(real_data, fake_data)

        # Combine losses (you can adjust the weights)
        alpha = 1.0  # Weight for perceptual loss
        beta = 1.0   # Weight for fidelity
        total_g_loss = g_loss + alpha * ploss - beta * torch.tensor(q_fidelity).to(device)

        total_g_loss.backward()
        g_optimizer.step()

        # Save losses and metrics
        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())
        perceptual_losses.append(ploss.item())
        fidelities.append(q_fidelity)

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}, '
          f'Perceptual Loss: {ploss.item():.4f}, Fidelity: {q_fidelity:.4f}')

Hyper parameter tuning

In [ ]:
def train_model(alpha, beta, num_epochs=10):
    # Define quantum devices
    dev_generator = qml.device('default.qubit', wires=n_qubits)
    dev_discriminator = qml.device('default.qubit', wires=n_qubits)

    # Define the generator and discriminator quantum circuits
    @qml.qnode(dev_generator, interface='torch')
    def generator_qnode(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]

    @qml.qnode(dev_discriminator, interface='torch')
    def discriminator_qnode(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return qml.expval(qml.PauliZ(0))

    # Define weight shapes for the models
    generator_weight_shapes = {"weights": (n_layers, n_qubits, 3)}
    discriminator_weight_shapes = {"weights": (n_layers, n_qubits, 3)}

    # Create the generator and discriminator models
    generator = qml.qnn.TorchLayer(generator_qnode, generator_weight_shapes).to(device)
    discriminator = qml.qnn.TorchLayer(discriminator_qnode, discriminator_weight_shapes).to(device)

    # Define optimizers and loss function
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.01)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.01)
    criterion = nn.BCELoss()

    # Lists to store metrics
    d_losses = []
    g_losses = []
    perceptual_losses = []
    fidelities = []
    total_g_losses = []

    start_time = time.time()

    for epoch in range(num_epochs):
        for i, real_data in enumerate(dataloader):
            batch_size = real_data.size(0)
            real_data = real_data.to(device)

            # Train Discriminator
            d_optimizer.zero_grad()
            # Real data
            real_labels = torch.ones(batch_size).to(device)
            real_output = discriminator(real_data)
            real_output = (real_output + 1)/2  # Map to [0,1]
            d_loss_real = criterion(real_output, real_labels)

            # Fake data
            noise = torch.randn(batch_size, n_qubits).to(device)
            fake_data = generator(noise)
            fake_data = fake_data.detach()
            fake_output = discriminator(fake_data)
            fake_output = (fake_output + 1)/2  # Map to [0,1]
            fake_labels = torch.zeros(batch_size).to(device)
            d_loss_fake = criterion(fake_output, fake_labels)

            # Total discriminator loss
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            noise = torch.randn(batch_size, n_qubits).to(device)
            fake_data = generator(noise)
            fake_output = discriminator(fake_data)
            fake_output = (fake_output + 1)/2  # Map to [0,1]
            g_loss = criterion(fake_output, real_labels)  # Try to fool discriminator

            # Compute Perceptual Loss
            ploss = perceptual_loss(real_data, fake_data)

            # Compute Quantum Fidelity
            q_fidelity = quantum_fidelity(real_data, fake_data)

            # Combine losses
            total_g_loss = g_loss + alpha * ploss - beta * torch.tensor(q_fidelity).to(device)

            total_g_loss.backward()
            g_optimizer.step()

            # Save losses and metrics
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            perceptual_losses.append(ploss.item())
            fidelities.append(q_fidelity)
            total_g_losses.append(total_g_loss.item())

        elapsed_time = time.time() - start_time
        print(f'Epoch [{epoch+1}/{num_epochs}], Time: {elapsed_time:.2f}s, '
              f'd_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}, '
              f'Total G Loss: {total_g_loss.item():.4f}, '
              f'Perceptual Loss: {ploss.item():.4f}, Fidelity: {q_fidelity:.4f}')

    # Return metrics for analysis
    return {
        'd_losses': d_losses,
        'g_losses': g_losses,
        'perceptual_losses': perceptual_losses,
        'fidelities': fidelities,
        'total_g_losses': total_g_losses,
        'generator': generator  # Return the trained generator
    }

In [ ]:
# Hyperparameter Grid Search over alpha and beta
alpha_values = [0.1, 1.0, 10.0]
beta_values = [0.1, 1.0, 10.0]
num_epochs = 10  # Adjust as needed

results = {}

for alpha in alpha_values:
    for beta in beta_values:
        print(f'\nTraining with alpha={alpha}, beta={beta}')
        metrics = train_model(alpha, beta, num_epochs=num_epochs)
        results[(alpha, beta)] = metrics

# Metrics Collection and Visualization
summary = []

for (alpha, beta), metrics in results.items():
    final_perceptual_loss = metrics['perceptual_losses'][-1]
    final_fidelity = metrics['fidelities'][-1]
    summary.append({
        'alpha': alpha,
        'beta': beta,
        'perceptual_loss': final_perceptual_loss,
        'fidelity': final_fidelity
    })

In [ ]:
df_summary = pd.DataFrame(summary)
print("\nSummary of Results:")
print(df_summary)

# Plotting Perceptual Loss vs. Fidelity for different alpha and beta values
plt.figure(figsize=(8,6))
for alpha in alpha_values:
    df_alpha = df_summary[df_summary['alpha'] == alpha]
    plt.plot(df_alpha['fidelity'], df_alpha['perceptual_loss'], marker='o', label=f'alpha={alpha}')
plt.xlabel('Quantum Fidelity')
plt.ylabel('Perceptual Loss')
plt.title('Perceptual Loss vs. Fidelity for Different Alpha Values')
plt.legend()
plt.show()

# Visualize Generator Outputs for Best Model
# Find the model with lowest perceptual loss
best_model_key = min(results, key=lambda x: results[x]['perceptual_losses'][-1])
best_generator = results[best_model_key]['generator']

print(f"\nBest model parameters: alpha={best_model_key[0]}, beta={best_model_key[1]}")

# Generate and visualize images using the best generator
noise = torch.randn(16, n_qubits).to(device)
fake_data = best_generator(noise)
fake_data = fake_data.detach().cpu().numpy()

# Inverse PCA to reconstruct images
fake_data_pca = pca.inverse_transform(fake_data)
fake_images = fake_data_pca.reshape(-1, 1, 16, 16)

# Plot generated images
plt.figure(figsize=(8,8))
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(fake_images[i, 0], cmap='gray')
    plt.axis('off')
plt.suptitle(f'Generated Images using Best Model (alpha={best_model_key[0]}, beta={best_model_key[1]})')
plt.tight_layout()
plt.show()